<strong>Date :</strong> Créé le 03 Avril 2021| Mis à jour le 09 Avril 2021 </strong>

<strong>Compétition Kaggle - Team Théo
    
@auteur : </strong>Théo SACCAREAU & Théo VEDIS

<strong>(2-1)_features_text_1
      
Description :</strong> A travers ce Notebook, nous détaillerons notre méthode pour créer les différentes features basées sur le texte.

Temps d'exécution du Notebook : environ  1h10.

# Installation / Téléchargement / Importation des librairies

In [ ]:
!pip install spacy 
!python -m spacy download en

In [ ]:
# Librairies usuelles
import pandas as pd
import numpy as np
from tqdm import tqdm 

# Librairies pour le texte
# (1) NLTK
import nltk
nltk.download('stopwords')
nltk.download('twitter_samples')
nltk.download('vader_lexicon')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.sentiment import SentimentIntensityAnalyzer

# (2) Spacy 
import spacy
import en_core_web_sm

# Chemin 

In [3]:
# Chemin relatif vers le dossier "data" (inutile de le changer).
pathFile = "../data/" 

# Chargement des données d'entrée

In [4]:
# Chargement du fichier contenant le DataFrame retourné par le Notebook précédent.
# Temps d'exécution : 1min30.
df = pd.read_json(pathFile + "df_clean.json") 

# (1) Traitement du texte. 
Dans cette première partie, nous effectuerons un nettoyage du texte : tokenisation, lemmentisation, suppression des mots-vides, etc. Nous en profiterons pour créer quelques features qui nous semblent être utiles pour la suite. 

## 1-1 Commentaires "robots" 
Pour commencer, nous allons vérifier que le contenu du commentaire n'est pas un commentaire "robot". Pour comprendre ce qu'on appelle "commentaire robot", il faut observer les commentaires qui reviennent le plus souvent : 

In [5]:
df["body"].value_counts()[:10]

En effet, en faisant ce `value_counts` nous nous sommes aperçu qu'il y avait quelques commentaires qui reviennent assez souvent et qui s'apparentent à des spams/commantaires bots. <br> 
Nous estimons alors qu'il peut être pertinent d'avoir une feature indiquant si oui ou non le commenataire est un "commentaire robot". Selon nous, si c'est le cas, le score ne sera pas élevé. 

Nous avons donc décider de considérer les commentaires qui reviennent plus de 50 fois et dont la taille est supérieure à 20 caractères (on fixe également une limite de caractère pour que les commentaires "Yes", "Thanks!", "No", ...,  qui reviennent souvent ne soient pas considérés comme des commentaires robots). 

In [6]:
# Seuils 
threshold_freq = 50
threshold_length = 20

# Value_counts
vc = df["body"].value_counts()

# Ensemble contenant les bodys considérés comme SPAM 
bot_comment = set([comment for comment, val in zip(
    tqdm(vc.index), vc.values) if val > threshold_freq and len(comment) > threshold_length])

100%|██████████| 3675455/3675455 [00:03<00:00, 920707.56it/s]


In [7]:
# On parcout la liste des contenus, si le contenu est dans l'ensemble précédent
# alors on met comme valeur 1, sinon 0.  
df['bot_comment'] = df['body'].apply(lambda x : 1 if x in bot_comment else 0)

In [8]:
sum(df['bot_comment'])

30960

Plus de 30 000, commentaires sont considérés comme des SPAM. 

## 1-2 Taille du commentaire avant NLP
Nous estimons que la taille du commentaire est importante. En effet, un commentaire très court (quelques caractères seulement), aura surement un score faible alors qu'un commentaire avec une phrase complète aura plus de chance d'avoir un bon score. Toutefois, nous pensons que la taille du commentaire ne doit pas être non plus trop grande car sinon les utilisateurs n'auront pas forcément envie de lire un gros paragraphe, et donc le score ne sera pas très élevé. 

Nous distinguons plusieurs types de taille : 
- la taille (en nombre de caractères) avant et après la suppression des stopwords (mots vides) et nettoyage du texte. Effectivement, si une phrase avait une longueur assez grande avant ce traitement et qu'après sa longueur devient très faible, alors forcément on se dit que la richesse du vocabulaire était pauvre (beaucoup de mots vides, etc). 
- la taille (nombre de mots) après le nettoyage du texte. 
- nombre de mots vides.

In [9]:
# Taille du body en nombre de caractères
df['length_comment_chars_before_NLP'] = [len(x) for x in tqdm(df['body'])]
df['length_comment_chars_before_NLP'][:10]

100%|██████████| 4234970/4234970 [00:02<00:00, 1697727.26it/s]


0    119
1     48
2      4
3     54
4    241
5     22
6    178
7      7
8     70
9    171
Name: length_comment_chars_before_NLP, dtype: int64

In [10]:
# Avec ce "tokenizer", on ne prend en compte que les caractères alpha-numérique, 
# la ponctuation est donc exclue.
tokenizer = nltk.RegexpTokenizer(r'\w+')

In [11]:
# Taille du body en nombre de mots avant le traitement du texte.
length_comment_words_before_NLP = [
    len(tokenizer.tokenize(x)) for x in tqdm(df['body'])]
    
length_comment_words_before_NLP[:10]

100%|██████████| 4234970/4234970 [00:31<00:00, 133626.49it/s]


[22, 9, 1, 15, 44, 3, 40, 2, 12, 32]

## 1-3 NLP 

### 1-3-1 Suppression des stopwords
L'objectif de cette étape est d'éliminer les "stopwords", c'est-à-dire les mots beaucoup utilisés mais peu informatifs et qui allourdissent notre jeux de données.

In [12]:
# Ensemble de mots-vides en anglais.
stop_words = set(tuple(nltk.corpus.stopwords.words('english')))

# On supprime les mots vides
df['bodyNLP1'] = [" ".join([i for i in tokenizer.tokenize(
    x.lower()) if not i in stop_words and len(i) >= 4]) for x in tqdm(df['body'])]

100%|██████████| 4234970/4234970 [00:55<00:00, 75929.24it/s]


In [13]:
# Body avant suppression des mots-vides.
df['body'][:5]

0    No one has a European accent either  because i...
1     That the kid ..reminds me of Kevin.   so sad :-(
2                                                 NSFL
3    I'm a guy and I had no idea this was a thing g...
4    Mid twenties male rocking skinny jeans/pants, ...
Name: body, dtype: object

In [14]:
# Body après suppression des mots-vides.
df['bodyNLP1'][:5]

0    european accent either exist accents europe eu...
1                                        reminds kevin
2                                                 nsfl
3                                      idea thing guys
4    twenties male rocking skinny jeans pants style...
Name: bodyNLP1, dtype: object

### 1-3-2 Lemmentisation ou racinisation  
C'est la dernière étape pour le prétraitement.

Le processus de « lemmatisation » consiste à représenter les mots sous leur forme canonique. Par exemple pour un verbe, ce sera son infinitif. Pour un nom, son masculin singulier. L'idée étant encore une fois de ne conserver que le sens des mots utilisés dans le corpus.

Il existe un autre processus qui exerce une fonction similaire qui s'appelle la racinisation (ou stemming en anglais). Cela consiste à ne conserver que la racine des mots étudiés. L'idée étant de supprimer les suffixes, préfixes et autres des mots afin de ne conserver que leur origine. 

Nous utiliserons ici la lemmentisation.

In [15]:
# Création d'un lemmatiseur
wnl = WordNetLemmatizer()

# Application de ce lemmatiseur au body après suppression des mots vides (NLP1)
df['bodyNLP2'] = pd.Series([" ".join([wnl.lemmatize(
    words) for words in tokenizer.tokenize(x)]) for x in tqdm(df['bodyNLP1'])])

100%|██████████| 4234970/4234970 [04:32<00:00, 15548.93it/s]


In [16]:
# Résultat après lemmentisation 
df['bodyNLP2'][:5]

0    european accent either exist accent europe eur...
1                                        reminds kevin
2                                                 nsfl
3                                       idea thing guy
4    twenty male rocking skinny jean pant styled ha...
Name: bodyNLP2, dtype: object

## 1-4 Taille du commentaire après NLP

In [17]:
# Taille du commentaire après suppression des mots-vides (NLP1) (caractères)
df['length_comment_chars_after_NLP'] = [len(x) for x in tqdm(df['bodyNLP1'])]
df['length_comment_chars_after_NLP'][:10]

100%|██████████| 4234970/4234970 [00:02<00:00, 1762469.91it/s]


0     59
1     13
2      4
3     15
4    164
5     22
6     82
7      0
8     50
9    122
Name: length_comment_chars_after_NLP, dtype: int64

In [18]:
# Taille du commentaire après suppression des mots-vides (mots)
df['length_comment_words'] = [len(tokenizer.tokenize(
    comment_after)) for comment_after in tqdm(df['bodyNLP1'])]
    
df['length_comment_words'][:10]

100%|██████████| 4234970/4234970 [00:16<00:00, 255522.01it/s]


0     8
1     2
2     1
3     3
4    24
5     3
6    13
7     0
8     7
9    18
Name: length_comment_words, dtype: int64

In [19]:
# On en déduit le nombre de mots vides (en faisant la soustraction du nombre de 
# mots avant et après le traitement)
df['nb_stopwords'] = [nb_before - nb_after for nb_before, nb_after in zip(
    tqdm(length_comment_words_before_NLP), df['length_comment_words'])]
    
df['nb_stopwords'][:10]

100%|██████████| 4234970/4234970 [00:01<00:00, 2168763.17it/s]


0    14
1     7
2     0
3    12
4    20
5     0
6    27
7     2
8     5
9    14
Name: nb_stopwords, dtype: int64

In [20]:
# Nombre de mots avant traitement du texte
sum(length_comment_words_before_NLP)

114581371

In [21]:
# Nombre de mots après traitement du texte
sum(df['length_comment_words'])

48111665

Avant le traitement, nous avions plus de 114 milliards de mots, après le traitement il n'y en a plus "que" 64 milliards. Près de 50% des mots étaients donc des mots vides ! 

# (2) Analyse d'opinions / de sentiments 
Dans cette deuxième partie, nous avons fait le choix d'effectuer une analyse de sentiments sur les commentaires. Cette analyse a permis de déterminer pour chaque commentaire s'il était plutôt "positif", "neutre", "negatif". Nous estimons que cette information peut être utile pour prédire le score d'un commentaire. <br> 
En effet, un commentaire "positif" aura selon nous plus de chance d'être bien noté. Toutefois, un commentaire "négatif" peut lui aussi bien être noté s'il répond à un sujet/post polémique auquel la majorité des utilisateurs de Reddit ne sont pas d'accord. Enfin, un commentaire "neutre" provoque selon nous moins de réaction.

In [22]:
def sentiment_cat(res):
    """
    Fonction qui permet à partir des résultats de l'analyseur de sentiments 
    de déterminer si le commentaire est positif, négatif ou neutre.
    Paramètre : 
        - res (float compris entre -1 et 1) : résultat de l'analyseur
    
    Sortie : 
        - sent (str) : chaine indiquant si le commentaire est "positif", 
        "négatif", ou "neutre". 
    """

    if (res > 0):
        sent = 'pos'
    elif (res == 0):
        sent = 'neu'
    elif (res <0):
        sent = 'neg'

    return sent

In [23]:
# Analyseur de sentiment 
sia = SentimentIntensityAnalyzer()

# Pour chaque contenu de commentaire, on applique l'analyseur puis on fait 
# appel à la fonction précédente pour déterminer son sentiment. 
df['sentiment'] = [sentiment_cat(sia.polarity_scores(
    x)['compound']) for x in tqdm(df['body'])]

100%|██████████| 4234970/4234970 [27:34<00:00, 2559.36it/s]


(!) Attention (!) Pour l'analyse de sentiments, il est mieux d'utiliser le corpus non nettoyé. En effet, les signes de ponctuations (!!), les émoticones ( :), :D, etc) sont des éléments que SIA prend en compte. Ils permettent donc d'améliorer les résultats.  

In [24]:
# Résultat 
df['sentiment'][:10]

0    neg
1    neg
2    neu
3    neg
4    pos
5    pos
6    pos
7    neu
8    pos
9    neg
Name: sentiment, dtype: object

A partir de ces résultats, on en déduit une nouvelle feature : les sentiments des enfants. <br> 
En considérant qu'un commentaire positif rapporte +1, qu'un commentaire négatif rapporte -1 et qu'un commentaire neutre rapporte 0, on fait la somme de tous les sentiments des enfants d'un commentaire. Cette somme peut être utile pour prédire un score. En effet, selon nous, plus cette somme est négative, plus les utilisateurs qui réponde à ce commentaire sont en désaccord et donc plus le commentaire aura tendance à avoir un score faible. A l'inverse, un commentaire où ses enfants sont majoritairement positifs, aura un score plus élevé. 


In [25]:
# On crée un dictionnaire qui contiendra la liste des sentiments de 
# leurs enfants.
dict_fils_sentiment = dict()

# On parcourt le DataFrame en récupérant les identifiants des parents 
# et les sentiments des commentaires. 
for parent, sentiment in zip(tqdm(df['parent_id']), df['sentiment']):

    # Si le parent est déjà dans le dictionnaire, on raajoute le sentiment 
    # de son nouvel enfant. 
    if parent in dict_fils_sentiment:
        dict_fils_sentiment[parent] = dict_fils_sentiment[parent] + [sentiment]
    
    # Sinon, c'est son premier, on créer donc une liste d'un élément contenant 
    # le sentiment de son premier enfant 
    else : 
        dict_fils_sentiment[parent] = [sentiment]

100%|██████████| 4234970/4234970 [00:15<00:00, 280766.16it/s]


In [26]:
def sentiment_child(liste):
    """
    Fonction qui traduit la liste des sentiments des enfants en somme pour 
    déterminer si les enfats sont plutôt positifs (somme > 0), neutre (somme =0)
    ou négative (somme < 0). 
    Paramètre : 
        - liste (liste) : liste des sentiments des enfants 
    
    Sortie : 
        - somme (int) : somme selon les sentiments des enfants.  
    """

    # Value_counts qui permet de connaitre la proportion de 'pos', 'neg' et 'neu'
    vc = pd.Series(liste).value_counts()

    # Dictionnaire permettant de convertir sentiment en entier
    dic = {
        'neu' : 0, 
        'pos' : 1, 
        'neg' : -1
    }

    return sum([dic[i] * val for i, val in zip(vc.index, vc.values)])

In [27]:
# On applique la fonction précédent à chaque commentaire
df['sentiment_child'] = [sentiment_child(
    dict_fils_sentiment[comment]) if comment in dict_fils_sentiment else 0 for comment in tqdm(df['name'])]

100%|██████████| 4234970/4234970 [23:36<00:00, 2990.09it/s]


In [28]:
df['sentiment_child'].value_counts()

 0      3341576
 1       471415
-1       313190
 2        44331
-2        26829
         ...   
 132          1
 140          1
 141          1
 144          1
 96           1
Name: sentiment_child, Length: 174, dtype: int64

Il y a une grande majorité de commantaire (3,34 millions) qui ont des enfants sans réaction particulière (somme =0). Cela s'explique par le fait que beaucoup de commentaire ne reçoivent pas de réponse (cf Notebooks suivants). <br>
Ensuite, le nombre de commantaires ayant des enfants positifs est globalement supérieur au nombre de commantaires ayant des enfants négatifs. 


# (3) Similarité avec le topic 
La dernière partie pour les features basées sur le texte porte sur la similarité entre le contenu du sujet/topic et le contenu du commentaire. Plus la similitude est proche, plus le commentaire traite du même sujet que le topic concerné. Ainsi, nous estimons que si la similitude est importante, le score du commentaire a de bonnes chances d'avoir un score élevé. <br> 
Nous nous sommes basé sur la métrique du cosinus. 


In [29]:
def embeddings_similarity(vector_a, vector_b):
    """
    Fonction qui calcule la similarité entre deux contenus textuels en se
    basant sur la méritque cosinus. 

    Paramètres : 
        - vector_a (list) : vecteur représentant le 1er contenu textuel 
        - vector_b (list) : vecteur représentant le 2e contenu textuel 

    Sortie : 
        - simi (float) : nombre décimal compris entre 0 et 1 indquant la 
        similarité entre les deux vectors (plus il est proche de 1 plus la 
        similarité est forte)
    """

    try:
        # Calcul de la similarité avec la formule du cosinus
        simi = np.dot(vec_1, vec_2) / \
            (np.linalg.norm(vec_1)*np.linalg.norm(vec_2))
    except:
        # S'il y a une erreur dans le calcul (notamment quand l'un des vecteurs
        # est nul parce que pas de contenu dans le body), on donne la valeur 0.
        simi = 0

    return simi

In [30]:
# Dans un premier temps, on récupère les informations scrapées sur les topics.
# En effet, dans notre DataFrame, nous ne disposions pas des contenus des posts
# nous les avons donc scrappés sur Reddit. 
# Temps exécution : 50s
df2 = pd.read_json(pathFile + "data_post.json").T 

In [ ]:
# Embeddins (version anglais) permettant de traduire un contenu textuel en
# vecteur.
embeddings = spacy.load('en_core_web_sm')

# On applique le même nettoyage au contenu des posts que celui effectué pour
# le contenu des commentaires (tokenisation suppresion mots vides et lemmentisation)
body_topics = df2['title'].apply(lambda x: " ".join(
    [i for i in tokenizer.tokenize(x.lower()) if not i in stop_words and len(i) >= 4]))
body_topics = [" ".join([wnl.lemmatize(words)
                         for words in tokenizer.tokenize(x)]) for x in tqdm(body_topics)]

# Une fois nettoyés, on transforme les contenus des posts en vecteur.
#body_topics = [embeddings(topic).vector for topic in tqdm(body_topics)]

In [ ]:
# On crée un dictionnaire qui associe à chaque topic leur vecteur représentant 
# leur contenu. 
dict_topic_body = dict([(topic, body) for topic, body in zip(tqdm(df2.index), body_topics)])

In [ ]:
# On applique la fonction permettant de calculer la similarité entre le contenu
# d'un commentaire et le contenu du post auquel il répond.

# df['cosSimWithTopic'] = [embeddings_similarity(embeddings(body).vector, dict_topic_body[topic])
#                         if topic in dict_topic_body else 0 for body, topic in zip(tqdm(df['bodyNLP2']), df['link_id'])]

Etant donné la longueur du DataFrame (4 millions de commentaires), cette fonction prend un temps trsè très important (près de 24h). Nous avons fait le choix, par soucis de temps, de ne pas utiliser cette fonction (bien que nous estimons après coup que cette feature aurait pu nous être d'une précieuse aide pour le modèle). 

# Sauvegarde 
La partie sur le texte n'est pas tout à fait fini, nous souhaitons réaliser également une analyse topicale. Cependant, nous ne pouvons pas la réaliser dans ce même Notebook car la RAM ne supportait pas toutes les opérations. Nous enregirtons donc les résultats dans un fichier et nous procéderons donc à l'analyse topicale dans le Notebook suivant. 

Pour ne pas surchager inutilement les fichiers de sauvegarde, nous supprimons les colonnes `body`, `bodyNLP1` et `bodyNLP2`. 

In [33]:
# On stocke le contenu nettoyé à part car on l'utilisera pour l'analyse topicale
data_LDA = df['bodyNLP2']

# Colonnes à supprimer 
columns_to_delete = ['body', 'bodyNLP1', 'bodyNLP2']

df = df.drop(columns=columns_to_delete)

In [34]:
# Sauvegarde DataFrame dans un fichier 
df.to_json(pathFile + "df_features_text.json")

In [35]:
# Sauvegarde du contenu des commentaires nettoyé
data_LDA.to_csv(pathFile + "data_LDA.csv", index=False)